In [1]:
from pathlib import Path
import pandas as pd
import requests
import zipfile
from tqdm.notebook import tqdm, trange
from lxml import etree

In [2]:
p = Path('.')
len(list(p.glob('data/*.zip')))

81

In [3]:
s3_url = 'https://s3.amazonaws.com/capitalbikeshare-data/'
r = requests.get(s3_url)
s3xml = etree.fromstring(r.content)
files = s3xml.xpath('/x:ListBucketResult/x:Contents/x:Key/text()', namespaces={'x': 'http://s3.amazonaws.com/doc/2006-03-01/'})
files[:5]

['2010-capitalbikeshare-tripdata.zip',
 '2011-capitalbikeshare-tripdata.zip',
 '2012-capitalbikeshare-tripdata.zip',
 '2013-capitalbikeshare-tripdata.zip',
 '2014-capitalbikeshare-tripdata.zip']

In [67]:
df = pd.DataFrame([f for f in files if f.endswith('.zip')], columns=['filename'])
df['date_prefix'] = df['filename'].str.split('-', n=1).str[0]
df['year'] = df['date_prefix'].str[:4]
df.tail()

,filename,date_prefix,year
77,202310-capitalbikeshare-tripdata.zip,202310,2023
78,202311-capitalbikeshare-tripdata.zip,202311,2023
79,202312-capitalbikeshare-tripdata.zip,202312,2023
80,202401-capitalbikeshare-tripdata.zip,202401,2024
81,202402-capitalbikeshare-tripdata.zip,202402,2024


In [43]:
def get_file(url, filename, path=Path('downloads')):
    if not path.is_dir():
        path.mkdir()
    r = requests.get(url + filename)
    open(path / filename, "wb").write(r.content)

def unzip_file(filename, path=Path('unzipped')):
    if not path.is_dir():
        path.mkdir()
    with zipfile.ZipFile(filename, 'r') as zip:
        if len(zip.infolist()) > 0:
            files = [f for f in zip.namelist() if not f.startswith('__MACOSX') and f.endswith('.csv')]
            if len(files) == 1:
                zip.extract(files[0], path)
            if len(files) > 1:
                tmp = []
                for f in files:
                    with zip.open(f) as csv:
                        tmp.append(pd.read_csv(csv))
                df = pd.concat(tmp)
                df.to_csv(path / filename[:-3] + 'csv', index=False)

In [8]:
'202112-capitalbikeshare-tripdata.zip'[:-3]+'csv'

'202112-capitalbikeshare-tripdata.csv'

In [5]:
with zipfile.ZipFile('data/202112-capitalbikeshare-tripdata.zip', 'r') as zip:
    csv = [f for f in zip.namelist() if not f.startswith('__MACOSX') and f.endswith('.csv')]
    print(len(csv))

1


In [6]:

path = Path('unzipped')
with zipfile.ZipFile('data/2012-capitalbikeshare-tripdata.zip', 'r') as zip:
    if len(zip.infolist()) > 0:
        files = [f for f in zip.namelist() if not f.startswith('__MACOSX') and f.endswith('.csv')]
        if len(files) == 1:
            zip.extract(files[0], path)
        if len(files) > 1:
            tmp = []
            for f in files:
                with zip.open(f) as csv:
                    tmp.append(pd.read_csv(csv))
            df = pd.concat(tmp)
            df.to_csv(path / '2012-capitalbikeshare-tripdata.csv', index=False)

In [ ]:
zip_dir = Path('downloads')
unzip_dir = Path('unzipped')
for file in tqdm(files):
    get_file(s3_url, file, path=zip_dir)
    unzip_file(zip_dir / file, path=unzip_dir)
print(f'Downloaded {len(list(unzip_dir.glob('*.csv')))} files.')